In [ ]:
import os
print(os.getcwd())
os.chdir('../')
print(os.getcwd())

In [ ]:
import numpy as np
import pandas as pd 
import wandb
api = wandb.Api()

In [ ]:
explanation_method_main=[["attention_last", 
                          "attention_rollout"],
                         ["gradcam", 
                          "gradcamgithub",
                          "igembedding",                                                              
                          "vanillaembedding",                          
                          "sgembedding",
                          "vargradembedding",
                          "LRP"],
                         ["leaveoneoutclassifier",
                          "riseclassifier", 
                          "ours"],
                        ["random"],
                        ]
explanation_method_main_flatten=[j for i in explanation_method_main for j in i]

dataset_name="ImageNette"
explanation_method=explanation_method_main_flatten[0]
gpus_classifier=3
backbone_type="vit_small_patch16_224"

In [ ]:
explanation_method

In [ ]:
!gpustat

In [ ]:
def get_retraining_status(backbone_type, dataset_name, api_dir="ch6845/transformer_interpretability_project_retraining"):
    api.flush()
    runs = api.runs(api_dir)
    result_dict_list=[]
    for run in runs:
        try:
            print(run.name)
            if run.config["datasets"]!=dataset_name:
                print(run.name, "pass")
                continue
            if run.config["backbone_type"]!=backbone_type:
                print(run.name, "pass")
                continue
            #print(run.config)
            result_dict_list.append({"name": run.name,
                                     "accuracy": run.summary["test/accuracy"] if "test/accuracy" in run.summary.keys() else np.nan,
                                     "cohenkappa": run.summary["test/cohenkappa"] if "test/cohenkappa" in run.summary.keys() else np.nan,
                                     "backbone_type": run.config["classifier_backbone_type"],
                                     "datasets": run.config["datasets"],
                                     "classifier_enable_pos_embed": run.config["classifier_enable_pos_embed"] if "classifier_enable_pos_embed" in run.config else True,

                                     "explanation_location_train": run.config["explanation_location_train"],
                                     "explanation_mask_amount_train": run.config["explanation_mask_amount_train"],
                                     "explanation_mask_ascending_train": run.config["explanation_mask_ascending_train"],

                                     "explanation_location_val": run.config["explanation_location_val"],
                                     "explanation_mask_amount_val": run.config["explanation_mask_amount_val"],
                                     "explanation_mask_ascending_val": run.config["explanation_mask_ascending_val"],

                                     "explanation_location_test": run.config["explanation_location_test"],
                                     "explanation_mask_amount_test": run.config["explanation_mask_amount_test"],
                                     "explanation_mask_ascending_test": run.config["explanation_mask_ascending_test"]

                                    })
        except:
            print(run.name, 'error')

        #print('\n')
    result_df=pd.DataFrame(result_dict_list)
    #result_df=result_df[(result_df["backbone_type"]==backbone_type) & (result_df["datasets"]==dataset_name)]
    return result_df

In [ ]:
for insert_delete in ["delete", "insert"]:
    for num_stage in [1, 3, 7, 14, 28, 56, 84, 112, 140, 168, 182]:
        exp_name=f"{dataset_name}_ROAR_classifier_{backbone_type}_1e-5_train_{insert_delete}_{explanation_method}_{num_stage}"
        
        classifier_enable_pos_embed=True
        
        explanation_location_train = f"/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/{dataset_name}/vit_base_patch16_224_{explanation_method}_train.pickle"
        explanation_mask_amount_train = 196-num_stage if insert_delete=="insert" else num_stage
        explanation_mask_ascending_train = True if insert_delete=="insert" else False
        
        explanation_location_val = f"/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/{dataset_name}/vit_base_patch16_224_{explanation_method}_val.pickle"
        explanation_mask_amount_val = 196-num_stage if insert_delete=="insert" else num_stage
        explanation_mask_ascending_val = True if insert_delete=="insert" else False
        
        explanation_location_test = f"/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/{dataset_name}/vit_base_patch16_224_{explanation_method}_test.pickle"
        explanation_mask_amount_test = 196-num_stage if insert_delete=="insert" else num_stage
        explanation_mask_ascending_test = True if insert_delete=="insert" else False
        
        retraining_status=get_retraining_status(backbone_type=backbone_type, 
                                                dataset_name=dataset_name,
                                                api_dir="ch6845/transformer_interpretability_project_retraining")
        
        if dataset_name=="ImageNette":
            checkpoint_metric="accuracy"
            loss_weight=''
        elif dataset_name=="MURA":
            checkpoint_metric="CohenKappa"
            loss_weight=" 'loss_weight = [21935, 14873]'"
        else:
            raise
        
        if len(retraining_status)==0 or not ((~retraining_status["accuracy"].isnull()) &\
                                             (retraining_status["classifier_enable_pos_embed"]==classifier_enable_pos_embed) &\
                                             (retraining_status["explanation_location_train"]==explanation_location_train) &\
                                             (retraining_status["explanation_mask_amount_train"]==explanation_mask_amount_train) &\
                                             (retraining_status["explanation_mask_ascending_train"]==explanation_mask_ascending_train) &\
                                             (retraining_status["explanation_location_val"]==explanation_location_val) &\
                                             (retraining_status["explanation_mask_amount_val"]==explanation_mask_amount_val) &\
                                             (retraining_status["explanation_mask_ascending_val"]==explanation_mask_ascending_val) &\
                                             (retraining_status["explanation_location_test"]==explanation_location_test) &\
                                             (retraining_status["explanation_mask_amount_test"]==explanation_mask_amount_test) &\
                                             (retraining_status["explanation_mask_ascending_test"]==explanation_mask_ascending_test)).any():
            command = f"~/miniconda3/envs/vitshapley/bin/python main.py with 'stage = \"classifier\"' \
            'wandb_project_name = \"transformer_interpretability_project_retraining\"' 'exp_name = \"{exp_name}\"' \
            env_chanwkim 'gpus_classifier=[{gpus_classifier}]' \
            dataset_{dataset_name}_ROAR \
            'classifier_enable_pos_embed=[{classifier_enable_pos_embed}]' \
            'explanation_location_train = \"{explanation_location_train}\"' 'explanation_mask_amount_train = {explanation_mask_amount_train}' 'explanation_mask_ascending_train = {explanation_mask_ascending_train}' \
            'explanation_location_val = \"{explanation_location_val}\"' 'explanation_mask_amount_val = {explanation_mask_amount_val}' 'explanation_mask_ascending_val = {explanation_mask_ascending_val}' \
            'explanation_location_test = \"{explanation_location_test}\"' 'explanation_mask_amount_test = {explanation_mask_amount_test}' 'explanation_mask_ascending_test = {explanation_mask_ascending_test}' \
            'classifier_backbone_type = \"{backbone_type}\"' 'classifier_download_weight = True' 'classifier_load_path = None' \
            training_hyperparameters_transformer{loss_weight} 'checkpoint_metric = \"{checkpoint_metric}\"' 'learning_rate = 1e-5'"
            print(command)
            !{command}



In [ ]:
for insert_delete in ["delete", "insert"]:
    for num_stage in ([1, 3, 7, 14, 28, 56, 84, 112, 140, 168, 182] if insert_delete=="insert" else [1, 3, 7, 14, 28, 56, 84, 112, 140, 168, 182][::-1]):
        exp_name=f"{dataset_name}_ROAR_classifier_{backbone_type}_1e-5_train_{insert_delete}_{explanation_method}_{num_stage}_nopos"
        
        classifier_enable_pos_embed=False
        
        explanation_location_train = f"/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/{dataset_name}/vit_base_patch16_224_{explanation_method}_train.pickle"
        explanation_mask_amount_train = 196-num_stage if insert_delete=="insert" else num_stage
        explanation_mask_ascending_train = True if insert_delete=="insert" else False
        
        explanation_location_val = f"/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/{dataset_name}/vit_base_patch16_224_{explanation_method}_val.pickle"
        explanation_mask_amount_val = 196-num_stage if insert_delete=="insert" else num_stage
        explanation_mask_ascending_val = True if insert_delete=="insert" else False
        
        explanation_location_test = f"/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/{dataset_name}/vit_base_patch16_224_{explanation_method}_test.pickle"
        explanation_mask_amount_test = 196-num_stage if insert_delete=="insert" else num_stage
        explanation_mask_ascending_test = True if insert_delete=="insert" else False
        
        retraining_status=get_retraining_status(backbone_type=backbone_type, 
                                                dataset_name=dataset_name,
                                                api_dir="ch6845/transformer_interpretability_project_retraining_nopos")
        
        if dataset_name=="ImageNette":
            checkpoint_metric="accuracy"
            loss_weight=''
        elif dataset_name=="MURA":
            checkpoint_metric="CohenKappa"
            loss_weight=" 'loss_weight = [21935, 14873]'"
        else:
            raise
        
        if len(retraining_status)==0 or not ((~retraining_status["accuracy"].isnull()) &\
                                             (retraining_status["classifier_enable_pos_embed"]==classifier_enable_pos_embed) &\
                                             (retraining_status["explanation_location_train"]==explanation_location_train) &\
                                             (retraining_status["explanation_mask_amount_train"]==explanation_mask_amount_train) &\
                                             (retraining_status["explanation_mask_ascending_train"]==explanation_mask_ascending_train) &\
                                             (retraining_status["explanation_location_val"]==explanation_location_val) &\
                                             (retraining_status["explanation_mask_amount_val"]==explanation_mask_amount_val) &\
                                             (retraining_status["explanation_mask_ascending_val"]==explanation_mask_ascending_val) &\
                                             (retraining_status["explanation_location_test"]==explanation_location_test) &\
                                             (retraining_status["explanation_mask_amount_test"]==explanation_mask_amount_test) &\
                                             (retraining_status["explanation_mask_ascending_test"]==explanation_mask_ascending_test)).any():
            command = f"~/miniconda3/envs/vitshapley/bin/python main.py with 'stage = \"classifier\"' \
            'wandb_project_name = \"transformer_interpretability_project_retraining_nopos\"' 'exp_name = \"{exp_name}\"' \
            env_chanwkim 'gpus_classifier=[{gpus_classifier}]' \
            dataset_{dataset_name}_ROAR \
            'classifier_enable_pos_embed={classifier_enable_pos_embed}' \
            'explanation_location_train = \"{explanation_location_train}\"' 'explanation_mask_amount_train = {explanation_mask_amount_train}' 'explanation_mask_ascending_train = {explanation_mask_ascending_train}' \
            'explanation_location_val = \"{explanation_location_val}\"' 'explanation_mask_amount_val = {explanation_mask_amount_val}' 'explanation_mask_ascending_val = {explanation_mask_ascending_val}' \
            'explanation_location_test = \"{explanation_location_test}\"' 'explanation_mask_amount_test = {explanation_mask_amount_test}' 'explanation_mask_ascending_test = {explanation_mask_ascending_test}' \
            'classifier_backbone_type = \"{backbone_type}\"' 'classifier_download_weight = True' 'classifier_load_path = None' \
            training_hyperparameters_transformer{loss_weight} 'checkpoint_metric = \"{checkpoint_metric}\"' 'learning_rate = 1e-5'"
            print(command)
            !{command}



In [ ]:
for insert_delete in ["insert"]:
    for num_stage in ([0]):
        exp_name=f"{dataset_name}_ROAR_classifier_{backbone_type}_1e-5_train_{insert_delete}_{explanation_method}_{num_stage}_nopos"
        
        classifier_enable_pos_embed=False
        
        explanation_location_train = f"/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/{dataset_name}/vit_base_patch16_224_{explanation_method}_train.pickle"
        explanation_mask_amount_train = 196-num_stage if insert_delete=="insert" else num_stage
        explanation_mask_ascending_train = True if insert_delete=="insert" else False
        
        explanation_location_val = f"/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/{dataset_name}/vit_base_patch16_224_{explanation_method}_val.pickle"
        explanation_mask_amount_val = 196-num_stage if insert_delete=="insert" else num_stage
        explanation_mask_ascending_val = True if insert_delete=="insert" else False
        
        explanation_location_test = f"/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/{dataset_name}/vit_base_patch16_224_{explanation_method}_test.pickle"
        explanation_mask_amount_test = 196-num_stage if insert_delete=="insert" else num_stage
        explanation_mask_ascending_test = True if insert_delete=="insert" else False
        
        retraining_status=get_retraining_status(backbone_type=backbone_type, 
                                                dataset_name=dataset_name,
                                                api_dir="ch6845/transformer_interpretability_project_retraining_nopos")
        
        if dataset_name=="ImageNette":
            checkpoint_metric="accuracy"
            loss_weight=''
        elif dataset_name=="MURA":
            checkpoint_metric="CohenKappa"
            loss_weight=" 'loss_weight = [21935, 14873]'"
        else:
            raise
        
        if len(retraining_status)==0 or not ((~retraining_status["accuracy"].isnull()) &\
                                             (retraining_status["classifier_enable_pos_embed"]==classifier_enable_pos_embed) &\
                                             (retraining_status["explanation_location_train"]==explanation_location_train) &\
                                             (retraining_status["explanation_mask_amount_train"]==explanation_mask_amount_train) &\
                                             (retraining_status["explanation_mask_ascending_train"]==explanation_mask_ascending_train) &\
                                             (retraining_status["explanation_location_val"]==explanation_location_val) &\
                                             (retraining_status["explanation_mask_amount_val"]==explanation_mask_amount_val) &\
                                             (retraining_status["explanation_mask_ascending_val"]==explanation_mask_ascending_val) &\
                                             (retraining_status["explanation_location_test"]==explanation_location_test) &\
                                             (retraining_status["explanation_mask_amount_test"]==explanation_mask_amount_test) &\
                                             (retraining_status["explanation_mask_ascending_test"]==explanation_mask_ascending_test)).any():
            command = f"~/miniconda3/envs/vitshapley/bin/python main.py with 'stage = \"classifier\"' \
            'wandb_project_name = \"transformer_interpretability_project_retraining_nopos\"' 'exp_name = \"{exp_name}\"' \
            env_chanwkim 'gpus_classifier=[{gpus_classifier}]' \
            dataset_{dataset_name}_ROAR \
            'classifier_enable_pos_embed={classifier_enable_pos_embed}' \
            'explanation_location_train = \"{explanation_location_train}\"' 'explanation_mask_amount_train = {explanation_mask_amount_train}' 'explanation_mask_ascending_train = {explanation_mask_ascending_train}' \
            'explanation_location_val = \"{explanation_location_val}\"' 'explanation_mask_amount_val = {explanation_mask_amount_val}' 'explanation_mask_ascending_val = {explanation_mask_ascending_val}' \
            'explanation_location_test = \"{explanation_location_test}\"' 'explanation_mask_amount_test = {explanation_mask_amount_test}' 'explanation_mask_ascending_test = {explanation_mask_ascending_test}' \
            'classifier_backbone_type = \"{backbone_type}\"' 'classifier_download_weight = True' 'classifier_load_path = None' \
            training_hyperparameters_transformer{loss_weight} 'checkpoint_metric = \"{checkpoint_metric}\"' 'learning_rate = 1e-5'"
            print(command)
            !{command}



~/miniconda3/envs/vitshapley/bin/python main.py with 'stage = "classifier"'             'wandb_project_name = "transformer_interpretability_project_retraining"' 'exp_name = "MURA_ROAR_classifier_vit_base_patch16_224_1e-5_train_original"'             env_chanwkim 'gpus_classifier=[7]'             dataset_MURA_ROAR             'explanation_location_train = None' 'explanation_mask_amount_train = None' 'explanation_mask_ascending_train = None'             'explanation_location_val = None' 'explanation_mask_amount_val = None' 'explanation_mask_ascending_val = None'             'explanation_location_test = None' 'explanation_mask_amount_test = None' 'explanation_mask_ascending_test = None'             'classifier_backbone_type = "vit_base_patch16_224"' 'classifier_download_weight = True' 'classifier_load_path = None'             training_hyperparameters_transformer 'loss_weight = [21935, 14873]' 'checkpoint_metric = "CohenKappa"' 'learning_rate = 1e-5'

In [ ]:
~/miniconda3/envs/vitshapley/bin/python main.py with 'stage = "classifier"'             'wandb_project_name = "transformer_interpretability_project_retraining_nopos"' 'exp_name = "ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_attention_last_0_nopos"'             env_chanwkim 'gpus_classifier=[3]'             dataset_ImageNette_ROAR             'classifier_enable_pos_embed=False'             'explanation_location_train = "/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/ImageNette/vit_base_patch16_224_attention_last_train.pickle"' 'explanation_mask_amount_train = 196' 'explanation_mask_ascending_train = True'             'explanation_location_val = "/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/ImageNette/vit_base_patch16_224_attention_last_val.pickle"' 'explanation_mask_amount_val = 196' 'explanation_mask_ascending_val = True'             'explanation_location_test = "/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/ImageNette/vit_base_patch16_224_attention_last_test.pickle"' 'explanation_mask_amount_test = 196' 'explanation_mask_ascending_test = True'             'classifier_backbone_type = "vit_base_patch16_224"' 'classifier_download_weight = True' 'classifier_load_path = None'             training_hyperparameters_transformer 'checkpoint_metric = "accuracy"' 'learning_rate = 1e-5'